# import stuff.

In [1]:
from os import environ
from dotenv import load_dotenv
import numpy as np
import pandas as pd
from riotwatcher import LolWatcher, ApiError

load_dotenv("../")

RIOT_API = environ["RIOT_API"]
lol_watcher = LolWatcher(RIOT_API)
my_region = "na1"
NA = "na1"

# initialize stuff

In [2]:
yogurtsauce = pd.DataFrame.from_dict(
    lol_watcher.summoner.by_name(NA, "yogurtsauce"), orient="index"
).T

ranked = lol_watcher.league.by_summoner(NA, yogurtsauce.id[0])
versions = lol_watcher.data_dragon.versions_for_region(NA)
champions_version = versions["n"]["champion"]
summoner_spells_version = versions["n"]["summoner"]
items_version = versions["n"]["item"]
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
my_matches = lol_watcher.match.matchlist_by_puuid(NA, yogurtsauce.puuid[0])
current_items = lol_watcher.data_dragon.items(items_version)
mastery_version = versions["n"]["mastery"]

# league

In [3]:
class League:
    def __init__(self, region) -> None:
        self.versions = self.Versions(region)
        self.champions = self.get_champions(region)

    class Versions:
        def __init__(self, region) -> None:
            self.versions_ = lol_watcher.data_dragon.versions_for_region(region)
            self.versions_item = self.versions_["n"]["item"]
            self.versions_rune = self.versions_["n"]["rune"]
            self.versions_mastery = self.versions_["n"]["mastery"]
            self.versions_summoner = self.versions_["n"]["summoner"]
            self.versions_champion = self.versions_["n"]["champion"]

    def get_champions(self, region):
        champions = (
            pd.DataFrame.from_dict(
                lol_watcher.data_dragon.champions(
                    self.Versions(region).versions_champion
                )
            )
            .reset_index()
            .rename(columns={"index": "champion"})
        )

        champions = (
            pd.concat([champions, pd.json_normalize(champions.data)], axis=1)
            .drop(columns=["type", "format", "version", "data", "name", "id"])
            .rename(
                {
                    "key": "champion_id",
                    "partype": "resource",
                    # info
                    "info.attack": "attack",
                    "info.defense": "defense",
                    "info.magic": "magic",
                    "info.difficulty": "difficulty",
                    # stats
                    # hp
                    "stats.hp": "hp",
                    "stats.hpperlevel": "hp_level",
                    "stats.hpregen": "hp_regen",
                    "stats.hpregenperlevel": "hpregen_level",
                    # mp
                    "stats.mp": "mp",
                    "stats.mpperlevel": "mp_level",
                    "stats.mpregen": "mp_regen",
                    "stats.mpregenperlevel": "mpregen_level",
                    # ad
                    "stats.attackdamage": "attack",
                    "stats.attackdamageperlevel": "attack_level",
                    # crit
                    "stats.crit": "crit",
                    "stats.critperlevel": "crit_level",
                    # armour
                    "stats.armor": "armor",
                    "stats.armorperlevel": "armor_level",
                    # attack speed
                    "stats.attackspeed": "atk_spd",
                    "stats.attackspeedperlevel": "atk_spd_level",
                    # ms
                    "stats.movespeed": "move_spd",
                    # attack range
                    "stats.attackrange": "atk_range",
                    # spellblock
                    "stats.spellblock": "spellblock",
                    "stats.spellblockperlevel": "spellblock_level",
                },
                axis=1,
            )
        )

        champions.insert(0, "champion_id", champions.pop("champion_id"))
        
        champions[["primary_role", "secondary_role"]] = pd.DataFrame(
            champions.tags.tolist(), index=champions.index
        )
        champions = champions.drop(columns="tags")
        
        # champions.primary_role = champions.primary_role.astype("category")
        # champions.secondary_role = champions.secondary_role.astype("category")
        champions.resource = champions.resource.astype("string")
        champions.champion_id = champions.champion_id.astype("int")

        return champions


League("na1").champions.head(1)

,champion_id,champion,title,blurb,resource,attack,defense,magic,difficulty,image.full,...,mp_regen,mpregen_level,crit,crit_level,attack,attack_level,atk_spd_level,atk_spd,primary_role,secondary_role
0,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,Blood Well,8,4,3,4,Aatrox.png,...,0.0,0.0,0,0,60,5.0,2.5,0.651,Fighter,Tank


# summoner

In [4]:
class Summoner:
    def __init__(self, region, summoner_name):
        self.region = region
        self.name = summoner_name
        self.data = pd.DataFrame.from_dict(
            lol_watcher.summoner.by_name(self.region, self.name), orient="index"
        ).T
        self.id = self.data.id[0]
        self.account_id = self.data.accountId[0]
        self.puuid = self.data.puuid[0]
        self.profile_icon_id = self.data.profileIconId[0]
        self.revision_date = self.data.revisionDate[0]
        self.level = self.data.summonerLevel[0]
        self.matches_ = lol_watcher.match.matchlist_by_puuid(self.region, self.puuid)


yogurtsauce = Summoner("na1", "yogurtsauce")

# champions

In [50]:
champions = (
    pd.DataFrame.from_dict(current_champ_list)
    .reset_index()
    .rename(columns={"index": "champion"})
)

champions = (
    pd.concat([champions, pd.json_normalize(champions.data)], axis=1)
    .drop(columns=["type", "format", "version", "data", "name", "id"])
    .rename(
        {
            "key": "champion_id",
            "partype": "resource",
            # info
            "info.attack": "info_attack",
            "info.defense": "info_defense",
            "info.magic": "info_magic",
            "info.difficulty": "info_difficulty",
            # stats
            # hp
            "stats.hp": "hp",
            "stats.hpperlevel": "hp_level",
            "stats.hpregen": "hp_regen",
            "stats.hpregenperlevel": "hpregen_level",
            # mp
            "stats.mp": "mp",
            "stats.mpperlevel": "mp_level",
            "stats.mpregen": "mp_regen",
            "stats.mpregenperlevel": "mpregen_level",
            # ad
            "stats.attackdamage": "attack",
            "stats.attackdamageperlevel": "attack_level",
            # crit
            "stats.crit": "crit",
            "stats.critperlevel": "crit_level",
            # armour
            "stats.armor": "armor",
            "stats.armorperlevel": "armor_level",
            # attack speed
            "stats.attackspeed": "atk_spd",
            "stats.attackspeedperlevel": "atk_spd_level",
            # ms
            "stats.movespeed": "move_spd",
            # attack range
            "stats.attackrange": "atk_range",
            # spellblock
            "stats.spellblock": "spellblock",
            "stats.spellblockperlevel": "spellblock_level",
        },
        axis=1,
    )
)

champions.insert(0, "champion_id", champions.pop("champion_id"))
champions[["primary_role", "secondary_role"]] = pd.DataFrame(
    champions.tags.tolist(), index=champions.index
)
champions = champions.drop(columns="tags")
# champions.primary_role = champions.primary_role.astype("category")
# champions.secondary_role = champions.secondary_role.astype("category")
# champions.resource = champions.resource.astype("string")
champions.champion_id = champions.champion_id.astype("int")

In [51]:
champions.head(1)

,champion_id,champion,title,blurb,resource,info_attack,info_defense,info_magic,info_difficulty,image.full,...,mp_regen,mpregen_level,crit,crit_level,attack,attack_level,atk_spd_level,atk_spd,primary_role,secondary_role
0,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,Blood Well,8,4,3,4,Aatrox.png,...,0.0,0.0,0,0,60,5.0,2.5,0.651,Fighter,Tank


In [60]:
champions[(champions["champion"] == "Aatrox")].resource.to_string(index=False)

'Blood Well'

# match history

In [38]:
def match_history(username, region, n_games=5):
    i = 0
    j = 0
    user = Summoner(region, username)
    matches = pd.DataFrame()

    while j < n_games:
        try:
            last_match = user.matches_[j]
            match_details = lol_watcher.match.by_id(region, last_match)
            match_players = match_details["info"]["participants"]
            details = []

            for player in match_players:
                dict = {}
                dict["ign"] = player["summonerName"]
                dict["level"] = player["summonerLevel"]
                dict["champion_id"] = player["championId"]
                dict["champion"] = player["championName"]
                dict[
                    "mastery_points"
                ] = lol_watcher.champion_mastery.by_summoner_by_champion(
                    region, player["summonerId"], player["championId"]
                )[
                    "championPoints"
                ]
                dict["team"] = player["teamId"]
                dict["win"] = player["win"]

                details.append(dict)

            details = pd.DataFrame.from_dict(details)
            details = details.merge(
                League("na1").champions[
                    [
                        "champion",
                        "attack",
                        "defense",
                        "magic",
                        "difficulty",
                        "resource",
                    ]
                ]
            )

            details.team = details.team.astype("category")

            matches = pd.concat([matches, details], ignore_index=True)

            i += 1
            j += 1
        except:
            j += 1

    return matches


test = match_history("yogurtsauce", "na1", 2)

In [100]:
test

,ign,level,champion_id,champion,mastery_points,team,win,attack,attack,defense,magic,difficulty,resource
0,bOqAlma,61,133,Quinn,485,100,False,9,59,4,2,5,Mana
1,Vastatio Tactics,477,59,JarvanIV,22134,100,False,6,64,8,3,5,Mana
2,yogurtsauce,522,107,Rengar,107505,100,False,7,68,4,2,8,Ferocity
3,FromDebris,389,57,Maokai,3990,100,False,3,64,8,6,3,Mana
4,Bewitching Poro,234,16,Soraka,21918,100,False,2,50,5,7,3,Mana
5,Joar Nessosin,639,6,Urgot,30084,200,True,8,63,5,3,8,Mana
6,RyanD871,365,54,Malphite,8867,200,True,5,62,9,7,2,Mana
7,PocketFulloClams,226,104,Graves,14443,200,True,8,68,5,3,3,Mana
8,MAGA 2024 USA,201,234,Viego,541,200,True,6,57,4,2,5,None
9,kingjimin99,45,110,Varus,3247,200,True,7,62,3,4,2,Mana


In [40]:
last_match = lol_watcher.match.by_id("na1", yogurtsauce.matches_[0])

In [41]:
temp_df = pd.DataFrame()

In [96]:
NUMBER_OF_ROWS = range(10)

red = pd.DataFrame()
blue = pd.DataFrame()

champion_columns = [(f"champ{i}") for i in range(1,6)]
level_columns = [(f"level{i}") for i in range(1,6)]
mastery_columns = [(f"mastery_level{i}") for i in range(1,6)]
resource_columns = [(f"resource{i}") for i in range(1,6)]

match_details = last_match["info"]["participants"]

for i in NUMBER_OF_ROWS:
    if i < 5:
        red.insert(i, champion_columns[i], [match_details[i]["championName"]])
        red.insert(i, level_columns[i], [match_details[i]["summonerLevel"]])
        red.insert(i, mastery_columns[i], [lol_watcher.champion_mastery.by_summoner_by_champion("na1", match_details[i]["summonerId"], match_details[i]["championId"])["championPoints"]])
        red.insert(i, resource_columns[i], [champions[(champions["champion"] == match_details[i]["championName"])].resource.to_string(index=False)])
    else:
        blue.insert(i - 5, champion_columns[i - 5], [match_details[i]["championName"]])
        blue.insert(i - 5, level_columns[i - 5], [match_details[i]["summonerLevel"]])
        blue.insert(i - 5, mastery_columns[i - 5], [lol_watcher.champion_mastery.by_summoner_by_champion("na1", match_details[i]["summonerId"], match_details[i]["championId"])["championPoints"]])
        blue.insert(i - 5, resource_columns[i - 5], [champions[(champions["champion"] == match_details[i]["championName"])].resource.to_string(index=False)])
        
red = red.reindex(sorted(red.columns), axis=1)
blue = blue.reindex(sorted(blue.columns), axis=1)
        
red.insert(0, "team", match_details[0]["teamId"])
red.insert(0, "win", match_details[0]["win"])

blue.insert(0, "team", match_details[5]["teamId"])
blue.insert(0, "win", match_details[5]["win"])

In [97]:
blue

,win,team,champ1,champ2,champ3,champ4,champ5,level1,level2,level3,...,mastery_level1,mastery_level2,mastery_level3,mastery_level4,mastery_level5,resource1,resource2,resource3,resource4,resource5
0,True,200,Urgot,Malphite,Graves,Viego,Varus,639,365,226,...,30084,8867,14443,541,3247,Mana,Mana,Mana,None,Mana


In [98]:
aram_matches = pd.DataFrame()

aram_matches = pd.concat([aram_matches, red, blue])
aram_matches

,win,team,champ1,champ2,champ3,champ4,champ5,level1,level2,level3,...,mastery_level1,mastery_level2,mastery_level3,mastery_level4,mastery_level5,resource1,resource2,resource3,resource4,resource5
0,False,100,Quinn,JarvanIV,Rengar,Maokai,Soraka,61,477,522,...,485,22134,107505,3990,21918,Mana,Mana,Ferocity,Mana,Mana
0,True,200,Urgot,Malphite,Graves,Viego,Varus,639,365,226,...,30084,8867,14443,541,3247,Mana,Mana,Mana,None,Mana
